In [2]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00


In [7]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.document_loaders.parsers import LanguageParser

In [5]:
## Clone the github repositories
%pwd
!mkdir test_repo
repo_path="test_repo"
Repo.clone_from("https://github.com/entbappy/End-to-end-ML-Project",to_path=repo_path)


<git.repo.base.Repo '/content/test_repo/.git'>

In [10]:
## loading the repo
repo_path="test_repo/"
loader=GenericLoader.from_filesystem(repo_path+'/src/mlProject',glob="**/*",suffixes=['.py'],
                                     parser=LanguageParser(language=Language.PYTHON))
documents=loader.load()
documents

[Document(metadata={'source': 'test_repo/src/mlProject/__init__.py', 'content_type': 'simplified_code', 'language': <Language.PYTHON: 'python'>}, page_content='import os\nimport sys\nimport logging\n\nlogging_str = "[%(asctime)s: %(levelname)s: %(module)s: %(message)s]"\n\nlog_dir = "logs"\n\nlog_filepath = os.path.join(log_dir, "running_log.log")\nos.makedirs(log_dir, exist_ok=True)\n\n\nlogging.basicConfig(\n    level = logging.INFO,\n    format = logging_str,\n\n    handlers=[\n        logging.FileHandler(log_filepath),\n        logging.StreamHandler(sys.stdout)\n    ]\n)\n\nlogger = logging.getLogger("mlProjectLogger")'),
 Document(metadata={'source': 'test_repo/src/mlProject/utils/common.py', 'content_type': 'functions_classes', 'language': <Language.PYTHON: 'python'>}, page_content='def read_yaml(path_to_yaml: Path) -> ConfigBox:\n    """reads yaml file and returns\n\n    Args:\n        path_to_yaml (str): path like input\n\n    Raises:\n        ValueError: if yaml file is empty\

In [12]:
## Chunkings
documents_splitter=RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON,
                                                                chunk_size=2000,
                                                                chunk_overlap=200)


texts=documents_splitter.split_documents(documents=documents)

In [15]:
## Embedding Model

In [17]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipython-input-17-3415835077.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 13.1 MB/s e

In [20]:
## Knowledge Base VectorDB
vectordb=Chroma.from_documents(texts,embedding=embeddings,persist_directory='./data')
vectordb.persist()


/tmp/ipython-input-20-2725352953.py:3: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [ ]:
## LLM Wrapper

#llm=ChatOpenAI()

In [24]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

pipe = pipeline("text-generation", model="tiiuae/falcon-7b-instruct", device=0)
llm = HuggingFacePipeline(pipeline=pipe)

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-24-2696712216.py:5: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [25]:
memory=ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)

/tmp/ipython-input-25-1926954105.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)


In [26]:
## QA Rapper
qa=ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type='mmr',search_kwargs={"k":3}),memory=memory)

In [27]:
## Q & A
question= "What is DataIngestion class?"

In [28]:
result=qa(question)
print(result['answer'])

/tmp/ipython-input-28-1805310032.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=qa(question)
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
# Code for: class DataIngestionConfig:



@dataclass(frozen=True)
# Code for: class DataValidationConfig:



@dataclass(frozen=True)
# Code for: class DataTransformationConfig:



@dataclass(frozen=True)
# Code for: class ModelTrainerConfig:



@dataclass(frozen=True)
# Code for: class ModelEvaluationConfig:

import os
from mlProject import logger
import pandas as pd
from mlProject.entity.config_entity import DataValidationConfig



# Code for: class DataValiadtion:

Question: What is DataIngestion class?
Helpful Answer:
The DataIngestion class is a configuration class that holds the necessary information for the application to re